<a href="https://colab.research.google.com/github/SubaruFallout/BasicOfDigitalImageProcessing/blob/master/constructionOfMorphologicalSkeleton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1. Построение морфологического скелета** <br>
Разработать программу построения морфологического скелета изображения и восстановления изображения по морфологическому скелету для бинарных изображений и различных структурирующих элементов.

Содержание отчета: <br>
&emsp;Исходные изображения <br>
&emsp;Код программы (для Jupiter Notebook) <br>
&emsp;Результат работы программы <br>
&emsp;Срок 01.11.2022 <br>

In [95]:
# import the necessary libraries
%matplotlib inline
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [97]:
# Skeleton construction algorithm
# Return number of iterations and skeletal subset
def constructSkelet(orig_img, struct_element):
    n = 0
    img = orig_img.copy()
    skeletons = []

    while True:
        eroded_img = cv2.erode(img, struct_element)
        if cv2.countNonZero(eroded_img) == 0:
            skeletons.append(img)
            break

        opened_img = cv2.dilate(eroded_img, struct_element)
        skeletons.append(cv2.subtract(img, opened_img))
        n += 1
        img = eroded_img.copy()

    return (n, skeletons)

In [96]:
# read image
img = cv2.imread("resouces/rose_binary.png", cv2.IMREAD_GRAYSCALE)
img = cv2.bitwise_not(img)

element = cv2.getStructuringElement(cv2.MORPH_CROSS, (3, 3))